In [ ]:
import os

import matplotlib.pyplot as plt

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.visualization import ImageNormalize, SqrtStretch

import sunpy.coordinates  # NOQA
import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a

from aiapy.calibrate.util import get_correction_table, get_pointing_table
from aiapy.calibrate import register, update_pointing, degradation, estimate_error

import pickle
import importlib
import numpy as np

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')


import all_nu_analysis as ana
import images_and_coalignment as iac

New keys by method:

Fit:

['17-nov-21_1', '06-sep-22', '22-nov-21_1']

Input:

['19-nov-21', '20-nov-21', '03-jun-22_2']

Double:

['09-dec-22']

In [ ]:
key = '27-jul-16_1'


###=========== new regions +++++++++++++++++++++++++++++++++++++++
#Use file 'new_aia_times.pickle'

#Load all of the new quiescent times.
with open('./reference_files/new_aia_times.pickle', 'rb') as f:
    data = pickle.load(f)


aagt = data['new_aia_times']


if key=='03-jun-22_2':
    pick_region=True
    
    regionind=0
    reg_inds = [8,9]
    shiftt = [-50, 50]
    #List of regions, within each region a list of ORBITS
    iti = [[aagt[8], aagt[9]]] #, [aagt[10], aagt[11]]]

    # regionind=1
    # reg_inds = [10,11]
    # shiftt = [-50, 50]
    # #List of regions, within each region a list of ORBITS
    # iti = [[aagt[10], aagt[11]]]

if key=='06-sep-22':
    pick_region=False
    reg_inds = [1,2]
    shiftt = [60, 70]
    iti = [aagt[1], aagt[2]]

if key=='22-nov-21_1':
    pick_region=False
    reg_inds = [6, 7]
    shiftt = [50, 50]
    iti = [aagt[6], aagt[7]]

if key=='19-nov-21':
    pick_region=False
    reg_inds = [3]
    shiftt = [50, -30]
    iti = [[aagt[3]]]

if key=='20-nov-21':
    pick_region=False
    reg_inds = [4, 5]
    shiftt = [70, -30]
    iti = [aagt[4], aagt[5]]


if key=='17-nov-21_1':
    pick_region=False
    reg_inds = [0]
    shiftt = [30, -70]
    iti = [aagt[0]]    
    

if key=='09-dec-22':
    pick_region=True

    # regionind=0
    # reg_inds = [12]
    # shiftt = [20, 80]
    # iti = [[aagt[12]]]  


    regionind=1
    reg_inds = [13]
    shiftt = [20, 80]
    iti = [[aagt[13]]]  

    # reg_inds = [12, 13]
    # shiftt = [20, 80]
    # iti = [[aagt[12]],[aagt[13]]]     


if key=='14-jan-21':
    pick_region=False
    reg_inds = [14, 15, 16, 17]
    shiftt = [60, 60]
    iti = [[aagt[14], aagt[15], aagt[16], aagt[17]]]



###=========== new regions +++++++++++++++++++++++++++++++++++++++
#Use file 'new_aia_times.pickle'

#Load all of the new quiescent times.
with open('./reference_files/oldnew_aia_times.pickle', 'rb') as f:
    data = pickle.load(f)


aagt = data['new_aia_times']


if key=='10-oct-17':
    pick_region=True
    
    regionind=0
    reg_inds = [2,3,4]
    shiftt = [20, 0]
    #List of regions, within each region a list of ORBITS
    iti = [[aagt[2], aagt[3], aagt[4]]] #, [aagt[10], aagt[11]]]

    # regionind=1
    # reg_inds = [5, 6,7]
    # shiftt = [20, 0]
    # #List of regions, within each region a list of ORBITS
    # iti = [[aagt[5], aagt[6], aagt[7]]]



if key=='26-jul-16_1':
    pick_region=True
    
    # regionind=0
    # reg_inds = [8,9,10]
    # shiftt = [-10, 0]
    # #List of regions, within each region a list of ORBITS
    # iti = [[aagt[8], aagt[9], aagt[10]]] #, [aagt[10], aagt[11]]]

    ##NOTE FIRST ONE IS EMPTY
    regionind=1
    reg_inds = [11,12,13]
    shiftt = [-10, 0]
    #List of regions, within each region a list of ORBITS
    iti = [[aagt[11], aagt[12], aagt[13]]]

if key=='26-jul-16_2':
    pick_region=False
    reg_inds = [14]
    shiftt = [-30, 20]
    iti = [[aagt[14]]]    


if key=='27-jul-16_1':
    pick_region=True
    
    regionind=0
    reg_inds = [15]
    shiftt = [-40, 20]
    iti = [[aagt[15]]]    


if key=='20-nov-21':
    pick_region=True
    
    regionind=0
    reg_inds = [17,18]
    shiftt = [60, 0]
    iti = [[aagt[17], aagt[18]]]    


print(iti)
go=True
    

file_ = '/Users/jmdunca2/do-dem/reference_files/all_targets_postghost.pickle'

with open(file_, 'rb') as f:
    data = pickle.load(f)
    
method = data[key]['method']
ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']
#print(len(obsids))


if method=='double':
    gauss_stats = ARDict['gauss_stats']
    sep_axis = gauss_stats[0][0]
else:
    sep_axis = ''

importlib.reload(iac)

regdicts=[]

print('')

if go:

    for i in range(0, len(id_dirs)):
        print(i)
        aiamaps=[]
    
        #Get saved orbit time from file.
        ind=reg_inds[i]
        ind2=0
        try:
            start_time = aagt[ind][ind2][0]
            print(start_time)
            
            jsoc_email='jessie.m.duncan@nasa.gov'
        
        
            #Get single full-disk 94 for region adjustment selection
            query = Fido.search(
                #a.Time(start_time+10*u.min, start_time + 11*u.min),
                a.Time(start_time, start_time + 10*u.s),
                a.Wavelength(94*u.angstrom),
                a.jsoc.Series.aia_lev1_euv_12s,
                a.jsoc.Notify(jsoc_email),
            )
            files = Fido.fetch(query)
        
            #Prep map
            amap=sunpy.map.Map(files[0])
            try:
                m = register(amap)
            except TypeError:
                amap.meta.pop('crpix1')
                amap.meta.pop('crpix2')
                print('CRPIX issue on ', files)
                m = register(amap)
        
            #Make list that is i long all containing the same map (the one for orbit i).
            for j in range(0, i+1):
                aiamaps.append(m)
            
            #Do region adjustment
            aati = iac.per_orbit_region_adjustment(working_dir, id_dirs, obsids, i, aiamaps, 
                                                   nushift=shiftt, method=method, shush=True, sep_axis=sep_axis, show=True,
                                                    pick_region=pick_region, regionind=regionind)
            #append new region dictionary to list
            if method == 'fit':
                regdicts.append(aati['aiaregdicts'][0])
            else:
                regdicts.append(aati['aiaregdicts'])

        except IndexError:
            regdicts.append([])
        

In [ ]:
#regdicts[0] = regdicts[1]
#regdicts[0] = [regdicts[0][1]]
regdicts

In [ ]:
importlib.reload(ana)

# ana.do_key_dem(key, file_, missing_last=False, plot_xrt=False, plot_aia=False,
#               high_temp_analysis=False, rscl=True,
#               aia_region_dict=regdicts, 
#               use_prepped_aia=False,
#               input_time_intervals=iti,
#               pick_region=pick_region, regionind=regionind,
#               do_no_xrt_version=False,
#               do_onlyaia_version=False,
#               do_aiaxrt_version=False)


ana.do_key_dem(key, file_, missing_last=False, plot_xrt=False, plot_aia=False,
              high_temp_analysis=False, rscl=True,
              aia_region_dict=regdicts, 
              use_prepped_aia=False,
              input_time_intervals=iti,
               pick_region=pick_region, regionind=regionind,
              do_no_xrt_version=True,
              do_onlyaia_version=False,
              do_aiaxrt_version=False)


ana.do_key_dem(key, file_, missing_last=False, plot_xrt=False, plot_aia=False,
              high_temp_analysis=False, rscl=True,
              aia_region_dict=regdicts, 
              use_prepped_aia=False,
              input_time_intervals=iti,
               pick_region=pick_region, regionind=regionind,
              do_no_xrt_version=False,
              do_onlyaia_version=True,
              do_aiaxrt_version=False)


ana.do_key_dem(key, file_, missing_last=False, plot_xrt=False, plot_aia=False,
              high_temp_analysis=False, rscl=True,
              aia_region_dict=regdicts, 
              use_prepped_aia=False,
              input_time_intervals=iti,
               pick_region=pick_region, regionind=regionind,
              do_no_xrt_version=False,
              do_onlyaia_version=False,
              do_aiaxrt_version=True)


In [ ]:
importlib.reload(ana)


ana.do_key_dem(key, file_, missing_last=False, plot_xrt=False, plot_aia=False,
              high_temp_analysis=True, rscl=True,
              aia_region_dict=regdicts, 
              use_prepped_aia=False,
              input_time_intervals=iti,
               pick_region=pick_region, regionind=regionind)


In [ ]:
file = '/Users/jmdunca2/do-dem/DEM_folders/initial_dem_10oct17/region_0/01-28-30_01-35-55/map_t17-10-10_01-28-23_prep_094.fits'
file = '/Users/jmdunca2/do-dem/DEM_folders/initial_dem_10oct17/region_0/aia.lev1_euv_12s.2017-10-10T013047Z.94.image.fits'
sunpy.map.Map(file)

In [ ]:
# ind=1
# ind2=0


# tr = aagt[ind][ind2] #[1]-aagt[ind][0]

# dur = (tr[1]-tr[0]).to(u.s)
# print(tr)

# start_time = tr[0]
# jsoc_email='jessie.m.duncan@nasa.gov'


# # #Get single full-disk 94 for region adjustment selection
# # query = Fido.search(
# #     a.Time(start_time, start_time + 10*u.s),
# #     a.Wavelength(94*u.angstrom),
# #     a.jsoc.Series.aia_lev1_euv_12s,
# #     a.jsoc.Notify(jsoc_email),
# # )
# # print(query)

# # files = Fido.fetch(query)
# # print(files)

In [ ]:
# # files = ['/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2021-11-17T172837Z.94.image_lev1.fits', 
# #          '/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2021-11-17T172837Z.94.spikes.fits', 
# #          '/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2021-11-17T172849Z.94.image_lev1.fits', 
# #          '/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2021-11-17T172849Z.94.spikes.fits']


# files = ['/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2022-09-06T172849Z.94.image_lev1.fits',
#         '/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2022-09-06T172901Z.94.image_lev1.fits']

# # files = ['/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2021-11-22T003237Z.94.image_lev1.fits',
# #         '/Users/jmdunca2/sunpy/data/aia.lev1_euv_12s.2021-11-22T003249Z.94.image_lev1.fits']

In [ ]:
# files

# amap=sunpy.map.Map(files)
# try:
#     m = register(amap)
# except TypeError:
#     amap.meta.pop('crpix1')
#     amap.meta.pop('crpix2')
#     print('CRPIX issue on ', files)
#     m = register(amap)

# aiamaps = [m]

In [ ]:
key='10-oct-17'
file_ = '/Users/jmdunca2/do-dem/reference_files/all_targets_postghost.pickle'

with open(file_, 'rb') as f:
    data = pickle.load(f)
    
method = data[key]['method']
ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

if method=='double':
    gauss_stats = ARDict['gauss_stats']
    sep_axis = gauss_stats[0][0]
else:
    sep_axis = ''



import glob
regionfiles = glob.glob(working_dir+'gauss_cen_'+obsids[0]+'_A_user_input*.reg')
regionfiles.sort()
print(regionfiles)

import region_fitting as rf

charstring='10oct17'

pfiles = glob.glob('/Users/jmdunca2/do-dem/DEM_folders//initial_dem_'+charstring+'/region_0/*6.0-10keV_min10time_intervals.pickle')
pfiles.sort()
print(pfiles)

ind=0

with open(pfiles[ind], 'rb') as f:
    timedata = pickle.load(f)

for i in range(0, len(timedata['time_intervals'])):
    time = timedata['time_intervals'][i]
    time0, time1 = time[0], time[1]

    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    print(timestring)

    regionfiles = glob.glob(working_dir+'gauss_cen_'+obsids[ind]+'_A_user_input*.reg')
    regionfiles.sort()
    #print(regionfiles)

    evt_file = '/Users/jmdunca2/do-dem/DEM_folders//initial_dem_'+charstring+'/region_0/'+timestring+'/nu'+obsids[ind]+'A06_0_4_p_cl_sunpos.evt'

    importlib.reload(rf)
    rf.plot_file_region(evt_file, time0, time1, regionfiles[0])

In [ ]:
regionfiles[0].split('.')[0].split('/')[-1]+'_'+timestring

In [ ]:
aati

In [ ]:
#Get single full-disk 94 for region adjustment selection
query = Fido.search(
    a.Time(start_time, start_time + 10*u.s),
    a.Wavelength(94*u.angstrom),
    a.jsoc.Series.aia_lev1_euv_12s,
    a.jsoc.Notify(jsoc_email),
)

files = Fido.fetch(query)

files.sort()

m=sunpy.map.Map(files[0])

ptab = get_pointing_table(m[0].date - 12 * u.h, m[0].date + 12 * u.h)
amap = update_pointing(m, pointing_table=ptab)

try:
    m = register(amap)
except TypeError:
    amap.meta.pop('crpix1')
    amap.meta.pop('crpix2')
    print('CRPIX issue on ', files)
    m = register(amap)




query = Fido.search(
    a.Time(start_time, start_time + dur),
    #a.Wavelength(171*u.angstrom),
    a.Wavelength(94*u.angstrom) | a.Wavelength(131*u.angstrom) | a.Wavelength(171*u.angstrom) | a.Wavelength(193*u.angstrom) | a.Wavelength(211*u.angstrom) | a.Wavelength(335*u.angstrom),
    a.Sample(10*u.s),
    a.jsoc.Series.aia_lev1_euv_12s,
    a.jsoc.Notify(jsoc_email),
    a.jsoc.Segment.image,
    cutout,
)

IN PROGRESS!!!

The issue here is that the aia dem prep built into dodem assumes that it will need to download the AIA data (full disk) using its own fido call. Whereas we have just downloaded that data here. There are two paths forward:

- Retain this method of getting the AIA cutouts, then add a line to copy them to their own location (dem directory?). From there, write an alternate aia_for_dem that uses the "checkfile" function from the nccs aia code to scan for cases where a data file is in the time range of interest, and then preps all of those.
- Add a Fido search option within aia_dem_prep that does cutouts based on the region (as done above). Then, use the rest of the existing local aia prep machinery as-is.

  Note that in both of these cases, it is important to review the NCCS aia code to check for changes with respect to the original aia prep code, and incorporate them for consistency. Notably, this includes deconvolution. Not sure what else.


In [ ]:
# blx = aati['aiaregdicts'][0]['centerx'].value-300
# bly = aati['aiaregdicts'][0]['centery'].value-300
# trx = aati['aiaregdicts'][0]['centerx'].value+300
# try_ = aati['aiaregdicts'][0]['centery'].value+300

# if np.any(np.abs([blx,bly,trx,try_]) > 1200):
#     print('WARNING BOX MAY GO OUT OF AIA FOV, CHECK YOUR CUTOUTS FOR WEIRD BEHAVIOR')

# bottom_left = SkyCoord(blx*u.arcsec, bly*u.arcsec, obstime=start_time, observer="earth", frame="helioprojective")
# top_right = SkyCoord(trx*u.arcsec, try_*u.arcsec, obstime=start_time, observer="earth", frame="helioprojective")



# cutout = a.jsoc.Cutout(bottom_left, top_right=top_right, tracking=True)

# jsoc_email = "jessie.m.duncan@nasa.gov"



#Get single full-disk 94 for region adjustment selection
# query = Fido.search(
#     a.Time(start_time, start_time + 10*u.s),
#     a.Wavelength(94*u.angstrom),
#     a.jsoc.Series.aia_lev1_euv_12s,
#     a.jsoc.Notify(jsoc_email),
# )

# files = Fido.fetch(query)

# files.sort()

# m=sunpy.map.Map(files[2])

# ptab = get_pointing_table(m.date - 12 * u.h, m.date + 12 * u.h)
# amap = update_pointing(m, pointing_table=ptab)

# try:
#     m = register(amap)
# except TypeError:
#     amap.meta.pop('crpix1')
#     amap.meta.pop('crpix2')
#     print('CRPIX issue on ', files)
#     m = register(amap)


# bottom_left2 = SkyCoord(blx*u.arcsec, bly*u.arcsec, frame=m.coordinate_frame)
# top_right2 = SkyCoord(trx*u.arcsec, try_*u.arcsec, frame=m.coordinate_frame)


# msub = m.submap(bottom_left=bottom_left2, top_right=top_right2)


# query = Fido.search(
#     a.Time(start_time, start_time + 3*u.min),
#     #a.Wavelength(171*u.angstrom),
#     a.Wavelength(94*u.angstrom) | a.Wavelength(131*u.angstrom) | a.Wavelength(171*u.angstrom) | a.Wavelength(193*u.angstrom) | a.Wavelength(211*u.angstrom) | a.Wavelength(335*u.angstrom),
#     a.Sample(10*u.s),
#     a.jsoc.Series.aia_lev1_euv_12s,
#     a.jsoc.Notify(jsoc_email),
#     a.jsoc.Segment.image,
#     cutout,
# )


# files_c = Fido.fetch(query)
# files_c.sort()

# aia_sequence = sunpy.map.Map(files_c, sequence=True)

# from sunpy.coordinates import propagate_with_solar_surface, Helioprojective

# aia_sequence_aligned_list = []
# with propagate_with_solar_surface():
#     for mm in aia_sequence:
#         with Helioprojective.assume_spherical_screen(mm.observer_coordinate):
#             nm = mm.reproject_to(msub.wcs)
#             nm.meta = mm.meta
#             aia_sequence_aligned_list.append(nm)


# aia_sequence_aligned = sunpy.map.Map(aia_sequence_aligned_list, sequence=True)
# #    aia_sequence_aligned = sunpy.map.Map([mm.reproject_to(msub.wcs) for mm in aia_sequence], sequence=True)

# for aa in aia_sequence_aligned:
#     wvn="{0:d}".format(1000+aa.meta['wavelnth'])
#     wvn=wvn[1:]
#     print('map_t'+aa.date.strftime('%y-%m-%d_%H-%M-%S')+'_prep_'+wvn+'.fits')
#     mm.save(filename, overwrite='True')



# query = Fido.search(
#     a.Time(start_time, start_time + dur),
#     #a.Wavelength(171*u.angstrom),
#     a.Wavelength(94*u.angstrom) | a.Wavelength(131*u.angstrom) | a.Wavelength(171*u.angstrom) | a.Wavelength(193*u.angstrom) | a.Wavelength(211*u.angstrom) | a.Wavelength(335*u.angstrom),
#     a.Sample(10*u.s),
#     a.jsoc.Series.aia_lev1_euv_12s,
#     a.jsoc.Notify(jsoc_email),
#     a.jsoc.Segment.image,
#     cutout,
# )
# print(query)